# 2024-11-01 이후 모든 데이터 가져오기

In [49]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import time

# =========================
# 1. 환경 설정 및 상수 정의
# =========================

# 필터 기준: 게시일 >= 2024-11-01
START_DATE_FILTER = datetime(2024, 11, 1)
START_DATE_REQUEST = START_DATE_FILTER.strftime("%Y-%m-%d") # "2024-11-01"

# 현재 날짜 (조회 종료일)
TODAY_DATE_REQUEST = datetime.now().strftime("%Y-%m-%d")

BASE_URL = "https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancList.do"
DETAIL_URL_BASE = "https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "Referer": "https://apply.lh.or.kr/"
}

# Payload 템플릿: 기간 필터 (2024-11-01부터 오늘까지) 적용, 유형 필터는 모두 제거
FORM_DATA_TEMPLATE = {
    'currPage': 1, 'panId': '', 'cnpCd': '', 'prevListCo': 50, 'srchAisTpCd': '', 'panSs': '',
    
    # 🚩 기간 필터: 시작일(2024-11-01) 설정, 종료일은 오늘
    'startDt': START_DATE_REQUEST, 
    'endDt': TODAY_DATE_REQUEST,  
    
    # 🚩 유형 필터 제거: 모든 유형의 데이터를 가져오기 위해 모두 비움
    'aisTpCd': '',          
    'uppAisTpCd': '',             
    'mi': '', 
    
    'srchUppAisTpCd': '', 'srchY': 'N', 'srchFilter': 'N', 'csCd': '', 'CNP_CD': '', 'ccrCnntSysDsCd': '', 'xssChk': 'N',
    'panEdDt': TODAY_DATE_REQUEST.replace('-', ''), # panEdDt도 설정
    'listCo': 50, 'panNm': '', 'indVal': 'N', 'maxSn': 50, 'mvinQf': '', 'netbgn': '', 'viewType': '',
    'panStDt': '', 
    'minSn': 0, 
    'schTy': '0', 
    'page': 1, 
}

def generate_detail_url(anchor_tag):
    """HTML 요소에서 상세 페이지 조회에 필요한 data 속성을 추출하여 URL을 생성합니다."""
    data_panId = anchor_tag.get('data-id1', '')
    data_ccr = anchor_tag.get('data-id2', '')
    data_upp = anchor_tag.get('data-id3', '')
    data_ais = anchor_tag.get('data-id4', '')
    
    if not all([data_panId, data_ccr, data_upp, data_ais]):
        return ""

    url = (
        f"{DETAIL_URL_BASE}&"
        f"panId={data_panId}&"
        f"ccrCnntSysDsCd={data_ccr}&" 
        f"uppAisTpCd={data_upp}&"     
        f"aisTpCd={data_ais}"         
    )
    return url

# =========================
# 2. 크롤링 핵심 함수
# =========================

def crawl_lh_notices_all_data():
    all_data = []
    page = 1
    list_count = FORM_DATA_TEMPLATE['listCo']
    
    print("🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...")
    print(f"**필터 기준: [모든 유형] + 게시일 {FORM_DATA_TEMPLATE['startDt']} 이후 데이터 수집.**")

    while True:
        form_data = FORM_DATA_TEMPLATE.copy()
        form_data['currPage'] = str(page)
        form_data['page'] = str(page) 
        form_data['minSn'] = str((page - 1) * list_count)
        form_data['maxSn'] = str(page * list_count)
        form_data['prevListCo'] = str(list_count)
        
        print(f"\n📄 Crawling page {page} ({form_data['minSn']} to {form_data['maxSn']})...")

        try:
            response = requests.post(BASE_URL, data=form_data, headers=HEADERS, timeout=15)
            response.raise_for_status() 

            soup = BeautifulSoup(response.text, 'html.parser')
            rows = soup.select("div.bbs_ListA table tbody tr")
            raw_data_count = len(rows)

            if raw_data_count == 0:
                print("🏁 데이터 없음. 목록의 끝에 도달했습니다. 크롤링 종료.")
                break
            
            new_data_count = 0
            
            for row in rows:
                try:
                    cols = [c.get_text(strip=True) for c in row.select("td")]
                    if len(cols) < 9: continue
                        
                    status = cols[7]
                    post_date_str = cols[5] 
                    
                    try:
                        post_date = datetime.strptime(post_date_str, '%Y.%m.%d') 
                        
                        # 🚩 핵심 로직: 2024-11-01 이전 데이터를 발견하면 즉시 루프 중단 (최신순 가정)
                        if post_date < START_DATE_FILTER:
                            print(f"🚩 게시일 {post_date_str} 데이터가 필터 기준({START_DATE_REQUEST})보다 이전입니다. 공고가 최신순이므로 추가 크롤링을 중단합니다.")
                            raw_data_count = 0 # 바깥 while 루프 종료 유도
                            break
                            
                    except ValueError:
                        # 날짜 형식이 아니면 일단 데이터를 수집하고 다음 행으로 넘어감
                        pass 
                        
                    title_anchor = row.select_one("td:nth-child(3) a")
                    em_tag = title_anchor.find('em')
                    if em_tag:
                        em_tag.extract()
                    title = title_anchor.get_text(strip=True).replace('\n', ' ')
                    
                    detail_url = generate_detail_url(title_anchor)
                    if not detail_url: continue 
                    
                    all_data.append({
                        '번호': cols[0], '유형': cols[1], '공고명': title, '지역': cols[3],
                        '게시일': post_date_str, '마감일': cols[6], '상태': status,
                        '조회수': cols[8].replace(',', ''), 'URL': detail_url
                    })
                    new_data_count += 1
                except Exception as e:
                    continue

            print(f"✅ 페이지 {page} 로드 성공. {len(rows)}개 중 {new_data_count}개 데이터 추출.")
            
            if raw_data_count == 0:
                 break
            
            if raw_data_count < list_count:
                 print("🏁 서버가 요청 개수보다 적은 항목을 반환했습니다. 목록의 끝으로 간주합니다. 크롤링 종료.")
                 break
            
            page += 1
            time.sleep(1) 

        except requests.exceptions.RequestException as e:
            print(f"🚨 네트워크 또는 HTTP 요청 오류 발생: {e}")
            break
        except Exception as e:
            print(f"🚨 치명적인 오류 발생: {e}")
            break

    # 4. 데이터 저장
    if all_data:
        save_to_csv(all_data)
    else:
        print(f"🎉 완료! 총 0개 저장됨")
        
def save_to_csv(data):
    """추출된 데이터를 CSV 파일로 저장합니다."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"lh_full_notices_{timestamp}.csv"
    
    fieldnames = ['번호', '유형', '공고명', '지역', '게시일', '마감일', '상태', '조회수', 'URL']
    
    try:
        with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)
        print(f"\n🎉 최종 완료! 총 {len(data)}개 저장됨 → {filename}")
    except Exception as e:
        print(f"❌ CSV 파일 저장 중 오류 발생: {e}")

# =========================
# 3. 실행
# =========================

if __name__ == "__main__":
    crawl_lh_notices_all_data()

🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...
**필터 기준: [모든 유형] + 게시일 2024-11-01 이후 데이터 수집.**

📄 Crawling page 1 (0 to 50)...
✅ 페이지 1 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 2 (50 to 100)...
✅ 페이지 2 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 3 (100 to 150)...
✅ 페이지 3 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 4 (150 to 200)...
✅ 페이지 4 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 5 (200 to 250)...
✅ 페이지 5 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 6 (250 to 300)...
✅ 페이지 6 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 7 (300 to 350)...
✅ 페이지 7 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 8 (350 to 400)...
✅ 페이지 8 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 9 (400 to 450)...
✅ 페이지 9 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 10 (450 to 500)...
✅ 페이지 10 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 11 (500 to 550)...
✅ 페이지 11 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 12 (550 to 600)...
✅ 페이지 12 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 13 (600 to 650)...
✅ 페이지 13 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 14 (650 to 70

### 1) 임대주택 분리

In [ ]:
import pandas as pd
from datetime import datetime

# =========================
# 1. 환경 설정 및 파일 경로 정의
# =========================

# 🚩 1. 저장된 CSV 파일명 (사용자가 제공한 파일명)
FILE_NAME = "lh_full_notices_20251118_145644.csv"

# 🚩 2. 필터링할 '임대주택' 유형 목록
LEASE_TYPES = [
    "통합공공임대",
    "통합공공임대(신혼희망)",
    "국민임대",
    "공공임대",
    "영구임대",
    "행복주택",
    "행복주택(신혼희망)",
    "장기전세",
    "신축다세대매입임대",
    "가정어린이집",
    "매입임대",
    "전세임대",
    "집주인임대",
    "6년 공공임대주택"
]

# =========================
# 2. 데이터 로드 및 필터링
# =========================

try:
    # CSV 파일을 로드합니다. '번호' 컬럼을 정수형으로 읽기 위해 dtype 지정
    df = pd.read_csv(FILE_NAME, dtype={'번호': int})
    print(f"✅ 파일 로드 성공: {FILE_NAME} (총 {len(df)}건)")

    # 1) '유형' 컬럼을 LEASE_TYPES 목록의 값들로 필터링합니다.
    df_filtered = df[df['유형'].isin(LEASE_TYPES)].copy()
    print(f"✅ '임대주택' 유형 필터링 완료: 총 {len(df_filtered)}건")

    if df_filtered.empty:
        print("🚨 필터링 결과, 해당하는 '임대주택' 데이터가 없습니다.")
    else:
        # =========================
        # 3. 데이터 정렬 및 ID 생성
        # =========================
        
        # '게시일' 컬럼을 datetime 객체로 변환합니다.
        # 주의: 오류가 발생할 수 있는 행은 무시합니다 (coerce).
        df_filtered['게시일_DT'] = pd.to_datetime(df_filtered['게시일'], format='%Y.%m.%d', errors='coerce')
        
        # 날짜 변환에 실패한 행(NaT)은 제거합니다.
        df_filtered.dropna(subset=['게시일_DT'], inplace=True)
        
        # 2) 정렬 기준: 
        #    1순위: 게시일_DT (과거 날짜 기준 오름차순)
        #    2순위: 번호 (오름차순)
        df_sorted = df_filtered.sort_values(
            by=['게시일_DT', '번호'],
            ascending=[True, True]
        ).reset_index(drop=True)

        # 3) 'ID' 컬럼 생성: LH_임대_{숫자}
        # 인덱스는 0부터 시작하므로, 1을 더하여 'LH_임대_1'부터 시작하도록 합니다.
        df_sorted['ID'] = 'LH_임대_' + (df_sorted.index + 1).astype(str)

        # =========================
        # 4. 최종 정리 및 저장 (파일 이름 수정 적용)
        # =========================
        
        # 최종 데이터프레임에서 '게시일_DT' 컬럼을 제거하고 'ID' 컬럼을 맨 앞으로 이동
        final_columns = ['ID'] + [col for col in df_sorted.columns if col not in ['ID', '게시일_DT']]
        df_final = df_sorted[final_columns]
        
        # 🚩 새 파일명 정의 (요청하신 형식)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file_name = f"lh_lease_notices_{timestamp}.csv"

        # 새 CSV 파일로 저장
        df_final.to_csv(output_file_name, index=False, encoding='utf-8-sig')
        
        print(f"\n🎉 작업 완료! 임대주택 데이터 {len(df_final)}건을 ID와 함께 저장했습니다.")
        print(f"새 파일명: {output_file_name}")

except FileNotFoundError:
    print(f"🚨 오류: 파일을 찾을 수 없습니다. 파일 이름 확인이 필요합니다: {FILE_NAME}")
except Exception as e:
    print(f"🚨 처리 중 예기치 않은 오류 발생: {e}")

✅ 파일 로드 성공: lh_full_notices_20251118_145644.csv (총 3009건)
✅ '임대주택' 유형 필터링 완료: 총 1643건

🎉 작업 완료! 임대주택 데이터 1643건을 ID와 함께 저장했습니다.
새 파일명: lh_lease_notices_20251118_150456.csv


### 2) 분양주택 분리

In [51]:
import pandas as pd
from datetime import datetime

# =========================
# 1. 환경 설정 및 파일 경로 정의
# =========================

# 🚩 1. 저장된 CSV 파일명 (사용자가 제공한 파일명)
FILE_NAME = "lh_full_notices_20251118_145644.csv"

# 🚩 2. 필터링할 '분양주택' 유형 목록
SALE_TYPES = [
    "분양주택",
    "공공분양(신혼희망)",
    "공공분양",
    # 추가적으로 토지, 상가 등 분양 관련 키워드가 있다면 여기에 포함합니다.
    # 만약 '토지', '상가'도 포함되어야 한다면 목록에 추가하세요.
]

# =========================
# 2. 데이터 로드 및 필터링
# =========================

try:
    # CSV 파일을 로드합니다. '번호' 컬럼을 정수형으로 읽기 위해 dtype 지정
    df = pd.read_csv(FILE_NAME, dtype={'번호': int})
    print(f"✅ 파일 로드 성공: {FILE_NAME} (총 {len(df)}건)")

    # 1) '유형' 컬럼을 SALE_TYPES 목록의 값들로 필터링합니다.
    df_filtered = df[df['유형'].isin(SALE_TYPES)].copy()
    print(f"✅ '분양주택' 유형 필터링 완료: 총 {len(df_filtered)}건")

    if df_filtered.empty:
        print("🚨 필터링 결과, 해당하는 '분양주택' 데이터가 없습니다.")
    else:
        # =========================
        # 3. 데이터 정렬 및 ID 생성
        # =========================
        
        # '게시일' 컬럼을 datetime 객체로 변환합니다. (정렬을 위해)
        df_filtered['게시일_DT'] = pd.to_datetime(df_filtered['게시일'], format='%Y.%m.%d', errors='coerce')
        
        # 날짜 변환에 실패한 행(NaT)은 제거합니다.
        df_filtered.dropna(subset=['게시일_DT'], inplace=True)
        
        # 2) 정렬 기준: 
        #    1순위: 게시일_DT (과거 날짜 기준 오름차순)
        #    2순위: 번호 (오름차순)
        df_sorted = df_filtered.sort_values(
            by=['게시일_DT', '번호'],
            ascending=[True, True]
        ).reset_index(drop=True)

        # 3) 'ID' 컬럼 생성: LH_분양_{숫자}
        # 인덱스는 0부터 시작하므로, 1을 더하여 'LH_분양_1'부터 시작하도록 합니다.
        df_sorted['ID'] = 'LH_분양_' + (df_sorted.index + 1).astype(str)

        # =========================
        # 4. 최종 정리 및 저장
        # =========================
        
        # 최종 데이터프레임에서 '게시일_DT' 컬럼을 제거하고 'ID' 컬럼을 맨 앞으로 이동
        final_columns = ['ID'] + [col for col in df_sorted.columns if col not in ['ID', '게시일_DT']]
        df_final = df_sorted[final_columns]
        
        # 🚩 새 파일명 정의 (요청하신 형식)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file_name = f"lh_sale_notices_{timestamp}.csv"

        # 새 CSV 파일로 저장
        df_final.to_csv(output_file_name, index=False, encoding='utf-8-sig')
        
        print(f"\n🎉 작업 완료! 분양주택 데이터 {len(df_final)}건을 ID와 함께 저장했습니다.")
        print(f"새 파일명: {output_file_name}")

except FileNotFoundError:
    print(f"🚨 오류: 파일을 찾을 수 없습니다. 파일 이름 확인이 필요합니다: {FILE_NAME}")
except Exception as e:
    print(f"🚨 처리 중 예기치 않은 오류 발생: {e}")

✅ 파일 로드 성공: lh_full_notices_20251118_145644.csv (총 3009건)
✅ '분양주택' 유형 필터링 완료: 총 164건

🎉 작업 완료! 분양주택 데이터 164건을 ID와 함께 저장했습니다.
새 파일명: lh_sale_notices_20251118_151128.csv


# 다운로드

### 1) 임대주택

In [79]:
import requests
import os
import re
import pandas as pd
import time
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import warnings

# 요청 시 SSL 인증서 검증 경고 비활성화
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

# ====================================================
# 🌟 설정 변수
# ====================================================
# 🚩 1. 입력 CSV 파일명
CSV_FILE = "lh_lease_notices_20251118_150456.csv"
# 🚩 2. 다운로드 폴더명
DOWNLOAD_DIR = "lh_downloads"
# 🚩 3. 테스트 모드 (0이면 전체 처리)
TEST_MODE_LIMIT = 0
# 🚩 4. 허용 확장자
ALLOWED_EXTENSIONS = ('.pdf', '.xls', '.xlsx')
# 🚩 5. LH 서버 인증 핵심 쿠키
FIXED_COOKIE_STRING = (
    "WMONID=Dn_IJtMsDIn; _gid=GA1.3.79854735.1763460967; _ga_6PG7CE7G5G=GS2.1.s1763460966$o8$g1$t1763460986$j40$l0$h1591851588; "
    "_ga=GA1.3.411649994.1763000357; JSESSIONID=x1eWdz56x2HLpWxU9Joduu6f_OfMRWq1NiMLTZXVm6WZlm9iRMUg!271591710!1038351822; "
    "quickMenuStts=Y; quickMenuSttsAuto=Y; NetFunnel_ID="
)

# LH 파일 다운로드 URL 패턴 및 정규식
LH_DOWNLOAD_PATH = "/lhapply/lhFile.do"
JS_PATTERN = re.compile(r"javascript:fileDownLoad\(['\"]?(\d+)['\"]?\);", re.IGNORECASE)
EXT_PATTERN = re.compile(r'\.([a-zA-Z0-9]+)$')


def parse_cookie_string(cookie_string):
    """쿠키 문자열을 딕셔너리로 변환"""
    cookies = {}
    for item in cookie_string.split(';'):
        if '=' in item:
            key, value = item.strip().split('=', 1)
            cookies[key] = value
    return cookies


def get_session_headers(referer_url, cookie_string):
    """요청에 필요한 헤더와 쿠키 딕셔너리를 생성"""
    fixed_user_agent = "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Mobile Safari/537.36"

    headers = {
        'User-Agent': fixed_user_agent,
        'Referer': referer_url,
        'Host': 'apply.lh.or.kr',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    cookies = parse_cookie_string(cookie_string)
    return headers, cookies


def sanitize_filename(name):
    """파일명 특수문자 제거 및 '바로가기' 텍스트 정리"""
    name = re.sub(r'[\\/:*?"<>|]', '_', name)
    name = re.sub(r'바\s*로\s*보\s*기\s*\d*|\s*바로가기\s*\d*', '', name, flags=re.IGNORECASE).strip()
    return name[:150].strip()


def download_file(download_url, file_id, pan_id, pan_name, referer_url, raw_filename):
    """첨부파일 하나를 다운로드합니다."""
    headers, cookies = get_session_headers(referer_url, FIXED_COOKIE_STRING)
    download_params = {'fileid': file_id}

    try:
        file_response = requests.get(
            download_url,
            params=download_params,
            headers=headers,
            cookies=cookies,
            stream=True,
            timeout=30,
            verify=False
        )
        file_response.raise_for_status()

        file_content = file_response.content
        actual_size = len(file_content)

        if actual_size < 1024:
            print(f"  ❌ 경고: 파일 다운로드 실패. 빈 응답 감지. (크기: {actual_size} bytes)")
            return False, 0, None

        file_ext_match = EXT_PATTERN.search(raw_filename.lower())
        file_ext = file_ext_match.group(0) if file_ext_match else ".dat"

        clean_name_base = raw_filename.replace(file_ext, '').strip()
        clean_name = sanitize_filename(clean_name_base)
        target_filename = f"({pan_id})_{clean_name}{file_ext}"

        save_path = os.path.join(DOWNLOAD_DIR, target_filename)
        collision = 1
        while os.path.exists(save_path):
            base, ext = os.path.splitext(target_filename)
            base_no_collision = re.sub(r'(\s*\(\d+\))?$', '', base)
            target_filename = f"{base_no_collision}({collision}){ext}"
            save_path = os.path.join(DOWNLOAD_DIR, target_filename)
            collision += 1

        with open(save_path, 'wb') as f:
            f.write(file_content)

        return True, actual_size, target_filename

    except requests.exceptions.RequestException as err:
        print(f"  ❌ 요청 오류: {err}")
        return False, 0, None
    except Exception as e:
        print(f"  ❌ 일반 오류: {e}")
        return False, 0, None


def process_notice(item, total_records, index):
    """단일 공고 URL에서 첨부파일을 추출하고 다운로드"""
    pan_id = item['ID']
    base_url = item['URL']
    pan_name = item['공고명']

    parsed_uri = urlparse(base_url)
    domain_base = f"{parsed_uri.scheme}://{parsed_uri.netloc}"
    download_url = f"{domain_base}{LH_DOWNLOAD_PATH}"

    print(f"\n--- {index}/{total_records}번째 공고 ({pan_id}: {pan_name}) 처리 중 ---")

    headers, cookies = get_session_headers(base_url, FIXED_COOKIE_STRING)
    try:
        response = requests.get(base_url, headers=headers, cookies=cookies, timeout=10, verify=False)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"  ❌ 공고 접속 실패: {e}")
        return 0

    soup = BeautifulSoup(response.content, 'html.parser')
    attachment_data = []

    for link in soup.find_all('a', href=True):
        href = link.get('href', '')
        match_js = JS_PATTERN.search(href)

        if match_js:
            file_id = match_js.group(1)
            raw_filename = link.text.strip()

            file_ext_match = EXT_PATTERN.search(raw_filename.lower())
            file_ext = file_ext_match.group(0) if file_ext_match else None

            if file_ext and file_ext in ALLOWED_EXTENSIONS:
                attachment_data.append({
                    'id': file_id,
                    'name': raw_filename
                })

    if not attachment_data:
        print("  ℹ️ 다운로드할 유효한 첨부파일을 찾지 못했습니다.")
        return 0

    files_on_this_page = 0
    for data in attachment_data:
        success, size, target_filename = download_file(
            download_url,
            data['id'],
            pan_id,
            pan_name,
            base_url,
            data['name']
        )
        if success:
            print(f"  ✅ 성공: {target_filename} (크기: {size:,} bytes)")
            files_on_this_page += 1

    print(f"  ✨ {files_on_this_page}개의 첨부파일 다운로드 시도 완료.")
    return files_on_this_page


def main():
    """메인 실행 함수"""
    start_time = time.time()

    try:
        df = pd.read_csv(CSV_FILE)
        all_url_data = df[['ID', 'URL', '공고명']].to_dict('records')
    except FileNotFoundError:
        print(f"🚨 오류: CSV 파일을 찾을 수 없습니다: {CSV_FILE}")
        return
    except Exception as e:
        print(f"🚨 CSV 파일 로드 중 오류 발생: {e}")
        return

    if not os.path.exists(DOWNLOAD_DIR):
        os.makedirs(DOWNLOAD_DIR)
        print(f"✅ 다운로드 폴더 생성 완료: {DOWNLOAD_DIR}")

    total_records = len(all_url_data)
    total_downloaded_count = 0

    print(f"\n🚀 총 {total_records}개의 공고 URL에 대해 파일 다운로드를 시작합니다.")

    for i, item in enumerate(all_url_data):
        downloaded_count = process_notice(item, total_records, i + 1)
        total_downloaded_count += downloaded_count
        time.sleep(0.5)

    end_time = time.time()
    print("\n============================================")
    print("✅ 전체 파일 다운로드 작업 완료")
    print(f"총 처리된 공고 URL 수: {total_records}개")
    print(f"총 다운로드 성공 파일 수: {total_downloaded_count}개")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print(f"다운로드 폴더: {DOWNLOAD_DIR}")
    print("============================================")


if __name__ == "__main__":
    main()


🚀 총 1643개의 공고 URL에 대해 파일 다운로드를 시작합니다.

--- 1/1643번째 공고 (LH_임대_1: [선계약후검증, 동호지정] 도청이전신도시 RH-11BL 국민임대주택 입주자 추가모집 공고('24.11.01)) 처리 중 ---
  ✅ 성공: (LH_임대_1)_도청이전신도시RH-11BL국민임대입주자모집공고('24.11.01).pdf (크기: 496,814 bytes)
  ✅ 성공: (LH_임대_1)_도청이전신도시RH-11BL국민_팜플렛.pdf (크기: 4,609,378 bytes)
  ✅ 성공: (LH_임대_1)_[■필독]공사양식작성방법.pdf (크기: 493,471 bytes)
  ✅ 성공: (LH_임대_1)_[■필독]제출서류안내.pdf (크기: 122,587 bytes)
  ✨ 4개의 첨부파일 다운로드 시도 완료.

--- 2/1643번째 공고 (LH_임대_2: 태백철암1 국민임대주택 예비입주자 모집 공고) 처리 중 ---
  ✅ 성공: (LH_임대_2)_2024.11.01태백철암국민임대주택모집공고문.pdf (크기: 1,090,409 bytes)
  ✨ 1개의 첨부파일 다운로드 시도 완료.

--- 3/1643번째 공고 (LH_임대_3: [선계약후검증, 동호지정] 도청이전신도시 RH10-2BL 국민임대주택 입주자 추가모집 공고('24.11.01)) 처리 중 ---
  ✅ 성공: (LH_임대_3)_도청이전신도시RH10-2BL국민임대입주자모집공고('24.11.01).pdf (크기: 503,683 bytes)
  ✅ 성공: (LH_임대_3)_도청이전신도시RH10-2국민_팜플렛.pdf (크기: 21,011,977 bytes)
  ✅ 성공: (LH_임대_3)_[■필독]공사양식작성방법.pdf (크기: 493,471 bytes)
  ✅ 성공: (LH_임대_3)_[■필독]제출서류안내.pdf (크기: 122,587 bytes)
  ✨ 4개의 첨부파일 다운로드 시도 완료.

--- 4/1643번째 공고 (LH_임대_4: 광주쌍촌 영구임대주택 예비입주자 모집 

- 'ID'와 '파일명 내에 포함된 ID'를 영어로 바꾸기

In [80]:
import pandas as pd
import os
import time

# ----------------------------------------------------
# 🌟 설정 변수
# ----------------------------------------------------
# 🚩 1. 임대 공고 CSV 파일 (한글 ID 포함)
ORIGINAL_CSV_FILE = "lh_lease_notices_20251118_150456.csv"
# 🚩 2. ID가 영어로 변경된 새 CSV 파일
NEW_CSV_FILE = "lh_lease_notices_eng.csv"
# 🚩 3. 다운로드 파일이 저장된 폴더
DOWNLOAD_DIR = "lh_downloads"

# 🚩 4. 변경할 ID 패턴 정의
KOREAN_ID_PREFIX = "LH_임대_"
ENGLISH_ID_PREFIX = "LH_lease_"
# ----------------------------------------------------

def rename_csv_id(original_file, new_file, old_prefix, new_prefix):
    """
    CSV 파일을 로드하여 'ID' 컬럼의 한글 ID를 영어 ID로 변경하고 새 파일로 저장합니다.
    """
    print(f"--- 1. CSV ID 변경 작업 시작 ({original_file} -> {new_file}) ---")
    try:
        # 1. 파일 로드
        df = pd.read_csv(original_file)
        
        # 2. 'ID' 컬럼에서 한글 접두사('LH_임대_')를 영어 접두사('LH_lease_')로 변경
        # .str.replace()는 pandas Series의 문자열 메소드입니다.
        df['ID'] = df['ID'].str.replace(old_prefix, new_prefix, regex=False)
        
        # 3. 새 파일로 저장 (한글 깨짐 방지를 위해 utf-8-sig 인코딩 사용)
        df.to_csv(new_file, index=False, encoding='utf-8-sig')
        
        print(f"✅ CSV ID 변경 및 저장 완료: {new_file}")
        return True
    
    except FileNotFoundError:
        print(f"🚨 오류: 원본 CSV 파일을 찾을 수 없습니다: {original_file}")
    except KeyError:
        print("🚨 오류: CSV 파일에 'ID' 컬럼이 없습니다.")
    except Exception as e:
        print(f"🚨 CSV 처리 중 예상치 못한 오류 발생: {e}")
    return False


def rename_downloaded_files(directory, old_prefix, new_prefix):
    """
    지정된 폴더 내의 모든 파일명을 순회하며 한글 ID 패턴을 영어 ID 패턴으로 변경합니다.
    (파일명에는 ID가 괄호로 감싸여 있으므로, 괄호 포함 패턴으로 치환합니다.)
    """
    print(f"\n--- 2. 다운로드 파일명 변경 작업 시작 ({directory} 폴더) ---")
    
    if not os.path.exists(directory):
        print(f"🚨 오류: 다운로드 폴더를 찾을 수 없습니다: {directory}")
        return 0

    # 파일명 패턴에 맞게 괄호를 포함하여 치환할 문자열 정의
    old_file_pattern = f"({old_prefix}" 
    new_file_pattern = f"({new_prefix}"
    
    renamed_count = 0
    
    for filename in os.listdir(directory):
        # 'LH_임대_' 패턴을 포함하는 파일만 대상으로 합니다.
        if old_file_pattern in filename:
            try:
                # 파일명에서 'LH_임대_' 부분을 'LH_lease_'로 치환
                new_filename = filename.replace(old_file_pattern, new_file_pattern)
                
                old_path = os.path.join(directory, filename)
                new_path = os.path.join(directory, new_filename)
                
                os.rename(old_path, new_path)
                renamed_count += 1
                # print(f"  ✅ 변경: {filename} -> {new_filename}") # 로그가 길어질 수 있어 주석 처리
            
            except Exception as e:
                print(f"  ❌ 파일명 변경 실패 ({filename}): {e}")

    print(f"✅ 파일명 변경 완료: 총 {renamed_count}개의 파일이 변경되었습니다.")
    return renamed_count


# --- 메인 실행 ---
if __name__ == "__main__":
    start_time = time.time()
    
    # 1. CSV ID 변경 및 새 파일 저장
    csv_success = rename_csv_id(
        ORIGINAL_CSV_FILE, 
        NEW_CSV_FILE, 
        KOREAN_ID_PREFIX, 
        ENGLISH_ID_PREFIX
    )
    
    if csv_success:
        # 2. 다운로드 파일명 변경
        rename_count = rename_downloaded_files(
            DOWNLOAD_DIR, 
            KOREAN_ID_PREFIX, 
            ENGLISH_ID_PREFIX
        )
    
    end_time = time.time()
    print("\n============================================")
    print("✨ ID 영문 변환 및 파일명 변경 작업 완료")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print("============================================")

--- 1. CSV ID 변경 작업 시작 (lh_lease_notices_20251118_150456.csv -> lh_lease_notices_eng.csv) ---
✅ CSV ID 변경 및 저장 완료: lh_lease_notices_eng.csv

--- 2. 다운로드 파일명 변경 작업 시작 (lh_downloads 폴더) ---
✅ 파일명 변경 완료: 총 2800개의 파일이 변경되었습니다.

✨ ID 영문 변환 및 파일명 변경 작업 완료
총 소요 시간: 1.19초


- '파일명' 컬럼 추가해서 별도로 lh_lease_notices-download.csv 만들기

In [81]:
import os
import re
import pandas as pd
import time

# ----------------------------------------------------
# 🌟 설정 변수
# ----------------------------------------------------
# 🚩 1. 다운로드 파일이 저장된 폴더
DOWNLOAD_DIR = "lh_downloads"

# 🚩 2. 처리할 공고의 ID 접두사 (LEASE)
PREFIX_TO_MAP = "LH_lease_"
# 🚩 3. 생성할 CSV 파일명 (LEASE)
OUTPUT_CSV_FILE = "lh_lease_notices-download.csv"

# ----------------------------------------------------

def create_mapping_csv(directory, prefix, output_file):
    """
    다운로드 폴더 내 파일명을 분석하여 ID와 파일명 매핑 CSV를 생성하고,
    ID의 숫자 순서대로 오름차순 정렬하여 저장합니다.
    """
    print(f"--- 1. {prefix} 파일 ID 매핑 작업 시작 (정렬 포함) ---")
    
    if not os.path.exists(directory):
        print(f"🚨 오류: 다운로드 폴더를 찾을 수 없습니다: {directory}")
        return 0

    # ID 전체 문자열을 추출하기 위한 정규 표현식 패턴
    # 예: (LH_lease_123)
    id_full_pattern = re.compile(rf'\({re.escape(prefix)}\d+\)')
    
    mapping_data = []
    mapped_count = 0
    
    for filename in os.listdir(directory):
        # 파일명이 ID 접두사를 포함하는지 확인합니다.
        if filename.startswith(f"({prefix}"):
            try:
                # 1. 파일명에서 ID 문자열을 찾습니다.
                match = id_full_pattern.match(filename)
                
                if match:
                    full_id_with_paren = match.group(0)
                    extracted_id = full_id_with_paren.strip('()')
                    
                    # 2. 정렬을 위해 ID에서 숫자 부분만 추출 (예: LH_lease_123 -> 123)
                    # ID가 항상 'PREFIX_숫자' 형식임을 가정합니다.
                    numeric_part = extracted_id.split('_')[-1]
                    numeric_id = int(numeric_part)
                    
                    # 3. 데이터 리스트에 추가 (정렬을 위한 임시 컬럼 포함)
                    mapping_data.append({
                        'ID': extracted_id,
                        '파일명': filename,
                        'SortIndex': numeric_id
                    })
                    mapped_count += 1
                
            except Exception as e:
                print(f"  ❌ 파일 분석 실패 ({filename}): {e}")

    if not mapping_data:
        print(f"⚠️ {prefix} 패턴에 일치하는 파일을 찾지 못했습니다. CSV 파일을 생성하지 않습니다.")
        return 0

    # 4. DataFrame 생성, 정렬 및 CSV 저장
    df = pd.DataFrame(mapping_data)
    
    # SortIndex(숫자)를 기준으로 오름차순 정렬
    df = df.sort_values(by='SortIndex', ascending=True)
    
    # 임시로 사용한 SortIndex 컬럼 제거
    df = df.drop(columns=['SortIndex'])
    
    try:
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"✅ ID 매핑 CSV 파일 생성 및 정렬 완료: {output_file}")
        print(f"   총 {mapped_count}개의 행이 기록되었습니다.")
        return mapped_count
    except Exception as e:
        print(f"🚨 CSV 파일 저장 중 오류 발생: {e}")
        return 0


# --- 메인 실행 ---
if __name__ == "__main__":
    start_time = time.time()
    
    # 1단계: 임대(lease) 파일 처리
    total_count = create_mapping_csv(
        DOWNLOAD_DIR, 
        PREFIX_TO_MAP, 
        OUTPUT_CSV_FILE
    )
    
    end_time = time.time()
    print("\n============================================")
    print("✨ ID 매핑 CSV 생성 작업 완료")
    print(f"처리된 파일 수: {total_count}개")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print("============================================")

--- 1. LH_lease_ 파일 ID 매핑 작업 시작 (정렬 포함) ---
✅ ID 매핑 CSV 파일 생성 및 정렬 완료: lh_lease_notices-download.csv
   총 2800개의 행이 기록되었습니다.

✨ ID 매핑 CSV 생성 작업 완료
처리된 파일 수: 2800개
총 소요 시간: 0.04초


### 2) 분양주택

In [ ]:
import requests
import os
import re
import pandas as pd
import time
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import warnings

# 요청 시 SSL 인증서 검증 경고 비활성화
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

# ====================================================
# 🌟 설정 변수
# ====================================================
# 🚩 1. 입력 CSV 파일명
CSV_FILE = "lh_sale_notices_20251118_151128.csv"
# 🚩 2. 다운로드 폴더명
DOWNLOAD_DIR = "lh_downloads"
# 🚩 3. 테스트 모드 (0이면 전체 처리)
TEST_MODE_LIMIT = 0
# 🚩 4. 허용 확장자
ALLOWED_EXTENSIONS = ('.pdf', '.xls', '.xlsx')
# 🚩 5. LH 서버 인증 핵심 쿠키
FIXED_COOKIE_STRING = (
    "WMONID=Dn_IJtMsDIn; _gid=GA1.3.79854735.1763460967; _ga_6PG7CE7G5G=GS2.1.s1763460966$o8$g1$t1763460986$j40$l0$h1591851588; "
    "_ga=GA1.3.411649994.1763000357; JSESSIONID=x1eWdz56x2HLpWxU9Joduu6f_OfMRWq1NiMLTZXVm6WZlm9iRMUg!271591710!1038351822; "
    "quickMenuStts=Y; quickMenuSttsAuto=Y; NetFunnel_ID="
)

# LH 파일 다운로드 URL 패턴 및 정규식
LH_DOWNLOAD_PATH = "/lhapply/lhFile.do"
JS_PATTERN = re.compile(r"javascript:fileDownLoad\(['\"]?(\d+)['\"]?\);", re.IGNORECASE)
EXT_PATTERN = re.compile(r'\.([a-zA-Z0-9]+)$')


def parse_cookie_string(cookie_string):
    """쿠키 문자열을 딕셔너리로 변환"""
    cookies = {}
    for item in cookie_string.split(';'):
        if '=' in item:
            key, value = item.strip().split('=', 1)
            cookies[key] = value
    return cookies


def get_session_headers(referer_url, cookie_string):
    """요청에 필요한 헤더와 쿠키 딕셔너리를 생성"""
    fixed_user_agent = "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Mobile Safari/537.36"

    headers = {
        'User-Agent': fixed_user_agent,
        'Referer': referer_url,
        'Host': 'apply.lh.or.kr',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    cookies = parse_cookie_string(cookie_string)
    return headers, cookies


def sanitize_filename(name):
    """파일명 특수문자 제거 및 '바로가기' 텍스트 정리"""
    name = re.sub(r'[\\/:*?"<>|]', '_', name)
    name = re.sub(r'바\s*로\s*보\s*기\s*\d*|\s*바로가기\s*\d*', '', name, flags=re.IGNORECASE).strip()
    return name[:150].strip()


def download_file(download_url, file_id, pan_id, pan_name, referer_url, raw_filename):
    """첨부파일 하나를 다운로드합니다."""
    headers, cookies = get_session_headers(referer_url, FIXED_COOKIE_STRING)
    download_params = {'fileid': file_id}

    try:
        file_response = requests.get(
            download_url,
            params=download_params,
            headers=headers,
            cookies=cookies,
            stream=True,
            timeout=30,
            verify=False
        )
        file_response.raise_for_status()

        file_content = file_response.content
        actual_size = len(file_content)

        if actual_size < 1024:
            print(f"  ❌ 경고: 파일 다운로드 실패. 빈 응답 감지. (크기: {actual_size} bytes)")
            return False, 0, None

        file_ext_match = EXT_PATTERN.search(raw_filename.lower())
        file_ext = file_ext_match.group(0) if file_ext_match else ".dat"

        clean_name_base = raw_filename.replace(file_ext, '').strip()
        clean_name = sanitize_filename(clean_name_base)
        target_filename = f"({pan_id})_{clean_name}{file_ext}"

        save_path = os.path.join(DOWNLOAD_DIR, target_filename)
        collision = 1
        while os.path.exists(save_path):
            base, ext = os.path.splitext(target_filename)
            base_no_collision = re.sub(r'(\s*\(\d+\))?$', '', base)
            target_filename = f"{base_no_collision}({collision}){ext}"
            save_path = os.path.join(DOWNLOAD_DIR, target_filename)
            collision += 1

        with open(save_path, 'wb') as f:
            f.write(file_content)

        return True, actual_size, target_filename

    except requests.exceptions.RequestException as err:
        print(f"  ❌ 요청 오류: {err}")
        return False, 0, None
    except Exception as e:
        print(f"  ❌ 일반 오류: {e}")
        return False, 0, None


def process_notice(item, total_records, index):
    """단일 공고 URL에서 첨부파일을 추출하고 다운로드"""
    pan_id = item['ID']
    base_url = item['URL']
    pan_name = item['공고명']

    parsed_uri = urlparse(base_url)
    domain_base = f"{parsed_uri.scheme}://{parsed_uri.netloc}"
    download_url = f"{domain_base}{LH_DOWNLOAD_PATH}"

    print(f"\n--- {index}/{total_records}번째 공고 ({pan_id}: {pan_name}) 처리 중 ---")

    headers, cookies = get_session_headers(base_url, FIXED_COOKIE_STRING)
    try:
        response = requests.get(base_url, headers=headers, cookies=cookies, timeout=10, verify=False)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"  ❌ 공고 접속 실패: {e}")
        return 0

    soup = BeautifulSoup(response.content, 'html.parser')
    attachment_data = []

    for link in soup.find_all('a', href=True):
        href = link.get('href', '')
        match_js = JS_PATTERN.search(href)

        if match_js:
            file_id = match_js.group(1)
            raw_filename = link.text.strip()

            file_ext_match = EXT_PATTERN.search(raw_filename.lower())
            file_ext = file_ext_match.group(0) if file_ext_match else None

            if file_ext and file_ext in ALLOWED_EXTENSIONS:
                attachment_data.append({
                    'id': file_id,
                    'name': raw_filename
                })

    if not attachment_data:
        print("  ℹ️ 다운로드할 유효한 첨부파일을 찾지 못했습니다.")
        return 0

    files_on_this_page = 0
    for data in attachment_data:
        success, size, target_filename = download_file(
            download_url,
            data['id'],
            pan_id,
            pan_name,
            base_url,
            data['name']
        )
        if success:
            print(f"  ✅ 성공: {target_filename} (크기: {size:,} bytes)")
            files_on_this_page += 1

    print(f"  ✨ {files_on_this_page}개의 첨부파일 다운로드 시도 완료.")
    return files_on_this_page


def main():
    """메인 실행 함수"""
    start_time = time.time()

    try:
        df = pd.read_csv(CSV_FILE)
        all_url_data = df[['ID', 'URL', '공고명']].to_dict('records')
    except FileNotFoundError:
        print(f"🚨 오류: CSV 파일을 찾을 수 없습니다: {CSV_FILE}")
        return
    except Exception as e:
        print(f"🚨 CSV 파일 로드 중 오류 발생: {e}")
        return

    if not os.path.exists(DOWNLOAD_DIR):
        os.makedirs(DOWNLOAD_DIR)
        print(f"✅ 다운로드 폴더 생성 완료: {DOWNLOAD_DIR}")

    total_records = len(all_url_data)
    total_downloaded_count = 0

    print(f"\n🚀 총 {total_records}개의 공고 URL에 대해 파일 다운로드를 시작합니다.")

    for i, item in enumerate(all_url_data):
        downloaded_count = process_notice(item, total_records, i + 1)
        total_downloaded_count += downloaded_count
        time.sleep(0.5)

    end_time = time.time()
    print("\n============================================")
    print("✅ 전체 파일 다운로드 작업 완료")
    print(f"총 처리된 공고 URL 수: {total_records}개")
    print(f"총 다운로드 성공 파일 수: {total_downloaded_count}개")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print(f"다운로드 폴더: {DOWNLOAD_DIR}")
    print("============================================")


if __name__ == "__main__":
    main()


✅ 다운로드 폴더 생성 완료: lh_downloads

🚀 총 164개의 공고 URL에 대해 파일 다운로드를 시작합니다.

--- 1/164번째 공고 (LH_분양_1: 양산사송 A-2블록 신혼희망타운(공공분양) 추가입주자 모집공고(잔금납부기간 변경)) 처리 중 ---
  ✅ 성공: (LH_분양_1)_양산사송에비뉴원신혼희망타운(공공분양)자격완화추가입주자모집공고문(선착순동호지정,잔금납부기간변경).pdf (크기: 372,890 bytes)
  ✅ 성공: (LH_분양_1)_양산사송A-2블록신혼희망타운(공공분양)팸플릿(공고용).pdf (크기: 44,673,662 bytes)
  ✅ 성공: (LH_분양_1)_공가세대옵션내역(홈페이지첨부).pdf (크기: 217,406 bytes)
  ✅ 성공: (LH_분양_1)_동호지정동호판(홈페이지등록,11월).pdf (크기: 104,660 bytes)
  ✨ 4개의 첨부파일 다운로드 시도 완료.

--- 2/164번째 공고 (LH_분양_2: 양주옥정 A-4(1)블록 공공분양주택 추가모집(5년 무이자할부 및 발코니 확장비 무상지원)) 처리 중 ---
  ✅ 성공: (LH_분양_2)_양주옥정A-4(1)BL공공분양주택추가입주자모집공고문(무이자할부)(241104)★.pdf (크기: 342,706 bytes)
  ✅ 성공: (LH_분양_2)_양주옥정A-4(1)BL팸플릿.pdf (크기: 7,005,694 bytes)
  ✅ 성공: (LH_분양_2)_양주옥정A-4(1)BL동호배치표(241104).pdf (크기: 187,413 bytes)
  ✨ 3개의 첨부파일 다운로드 시도 완료.

--- 3/164번째 공고 (LH_분양_3: 전주시(풍경빌) 임대주택 분양전환 후 잔여세대 일반매각 공고) 처리 중 ---
  ✅ 성공: (LH_분양_3)_(요약)전주시(풍경빌)임대주택분양전환후잔여세대일반매각공고.pdf (크기: 94,810 bytes)
  ✅ 성공: (LH_분양_3)_전주시(풍경빌)임대주택분양전환후잔여세대일반매각동호지정공고.pdf (크기: 294,2

- 'ID'와 '파일명 내에 포함된 ID'를 영어로 바꾸기

In [77]:
import pandas as pd
import os
import time

# ----------------------------------------------------
# 🌟 설정 변수
# ----------------------------------------------------
# 🚩 1. 분양 공고 CSV 파일 (한글 ID 포함)
ORIGINAL_CSV_FILE = "lh_sale_notices_20251118_151128.csv"
# 🚩 2. ID가 영어로 변경된 새 CSV 파일
NEW_CSV_FILE = "lh_sale_notices_eng.csv"
# 🚩 3. 다운로드 파일이 저장된 폴더
DOWNLOAD_DIR = "lh_downloads"

# 🚩 4. 변경할 ID 패턴 정의 (LH_분양_ -> LH_sale_)
KOREAN_ID_PREFIX = "LH_분양_"
ENGLISH_ID_PREFIX = "LH_sale_"
# ----------------------------------------------------

def rename_csv_id(original_file, new_file, old_prefix, new_prefix):
    """
    CSV 파일을 로드하여 'ID' 컬럼의 한글 ID를 영어 ID로 변경하고 새 파일로 저장합니다.
    """
    print(f"--- 1. CSV ID 변경 작업 시작 ({original_file} -> {new_file}) ---")
    try:
        # 1. 파일 로드
        df = pd.read_csv(original_file)
        
        # 2. 'ID' 컬럼에서 한글 접두사('LH_분양_')를 영어 접두사('LH_sale_')로 변경
        # .str.replace()는 pandas Series의 문자열 메소드입니다.
        df['ID'] = df['ID'].str.replace(old_prefix, new_prefix, regex=False)
        
        # 3. 새 파일로 저장 (한글 깨짐 방지를 위해 utf-8-sig 인코딩 사용)
        df.to_csv(new_file, index=False, encoding='utf-8-sig')
        
        print(f"✅ CSV ID 변경 및 저장 완료: {new_file}")
        return True
    
    except FileNotFoundError:
        print(f"🚨 오류: 원본 CSV 파일을 찾을 수 없습니다: {original_file}")
    except KeyError:
        print("🚨 오류: CSV 파일에 'ID' 컬럼이 없습니다.")
    except Exception as e:
        print(f"🚨 CSV 처리 중 예상치 못한 오류 발생: {e}")
    return False


def rename_downloaded_files(directory, old_prefix, new_prefix):
    """
    지정된 폴더 내의 모든 파일명을 순회하며 한글 ID 패턴을 영어 ID 패턴으로 변경합니다.
    (파일명에는 ID가 괄호로 감싸여 있으므로, 괄호 포함 패턴으로 치환합니다.)
    """
    print(f"\n--- 2. 다운로드 파일명 변경 작업 시작 ({directory} 폴더) ---")
    
    if not os.path.exists(directory):
        print(f"🚨 오류: 다운로드 폴더를 찾을 수 없습니다: {directory}")
        return 0

    # 파일명 패턴에 맞게 괄호를 포함하여 치환할 문자열 정의
    # 예: (LH_분양_ -> (LH_sale_
    old_file_pattern = f"({old_prefix}" 
    new_file_pattern = f"({new_prefix}"
    
    renamed_count = 0
    
    for filename in os.listdir(directory):
        # 'LH_분양_' 패턴을 포함하는 파일만 대상으로 합니다.
        if old_file_pattern in filename:
            try:
                # 파일명에서 'LH_분양_' 부분을 'LH_sale_'로 치환
                new_filename = filename.replace(old_file_pattern, new_file_pattern)
                
                old_path = os.path.join(directory, filename)
                new_path = os.path.join(directory, new_filename)
                
                os.rename(old_path, new_path)
                renamed_count += 1
                # print(f"  ✅ 변경: {filename} -> {new_filename}") # 로그가 길어질 수 있어 주석 처리
            
            except Exception as e:
                print(f"  ❌ 파일명 변경 실패 ({filename}): {e}")

    print(f"✅ 파일명 변경 완료: 총 {renamed_count}개의 파일이 변경되었습니다.")
    return renamed_count


# --- 메인 실행 ---
if __name__ == "__main__":
    start_time = time.time()
    
    # 1. CSV ID 변경 및 새 파일 저장
    csv_success = rename_csv_id(
        ORIGINAL_CSV_FILE, 
        NEW_CSV_FILE, 
        KOREAN_ID_PREFIX, 
        ENGLISH_ID_PREFIX
    )
    
    if csv_success:
        # 2. 다운로드 파일명 변경
        rename_count = rename_downloaded_files(
            DOWNLOAD_DIR, 
            KOREAN_ID_PREFIX, 
            ENGLISH_ID_PREFIX
        )
    
    end_time = time.time()
    print("\n============================================")
    print("✨ ID 영문 변환 및 파일명 변경 작업 완료")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print("============================================")

--- 1. CSV ID 변경 작업 시작 (lh_sale_notices_20251118_151128.csv -> lh_sale_notices_eng.csv) ---
✅ CSV ID 변경 및 저장 완료: lh_sale_notices_eng.csv

--- 2. 다운로드 파일명 변경 작업 시작 (lh_downloads 폴더) ---
✅ 파일명 변경 완료: 총 412개의 파일이 변경되었습니다.

✨ ID 영문 변환 및 파일명 변경 작업 완료
총 소요 시간: 0.11초


- '파일명' 컬럼 추가해서 별도로 lh_sale_notices-download.csv 만들기

In [78]:
import os
import re
import pandas as pd
import time

# ----------------------------------------------------
# 🌟 설정 변수
# ----------------------------------------------------
# 🚩 1. 다운로드 파일이 저장된 폴더
DOWNLOAD_DIR = "lh_downloads"

# 🚩 2. 처리할 공고의 ID 접두사 (SALE로 변경)
PREFIX_TO_MAP = "LH_sale_"
# 🚩 3. 생성할 CSV 파일명 (SALE로 변경)
OUTPUT_CSV_FILE = "lh_sale_notices-download.csv"

# ----------------------------------------------------

def create_mapping_csv(directory, prefix, output_file):
    """
    다운로드 폴더 내 파일명을 분석하여 ID와 파일명 매핑 CSV를 생성하고,
    ID의 숫자 순서대로 오름차순 정렬하여 저장합니다.
    """
    print(f"--- 1. {prefix} 파일 ID 매핑 작업 시작 (정렬 포함) ---")
    
    if not os.path.exists(directory):
        print(f"🚨 오류: 다운로드 폴더를 찾을 수 없습니다: {directory}")
        return 0

    # ID 전체 문자열을 추출하기 위한 정규 표현식 패턴
    # 예: (LH_sale_123)
    id_full_pattern = re.compile(rf'\({re.escape(prefix)}\d+\)')
    
    mapping_data = []
    mapped_count = 0
    
    for filename in os.listdir(directory):
        # 파일명이 ID 접두사를 포함하는지 확인합니다.
        if filename.startswith(f"({prefix}"):
            try:
                # 1. 파일명에서 ID 문자열을 찾습니다.
                match = id_full_pattern.match(filename)
                
                if match:
                    full_id_with_paren = match.group(0)
                    extracted_id = full_id_with_paren.strip('()')
                    
                    # 2. 정렬을 위해 ID에서 숫자 부분만 추출 (예: LH_sale_123 -> 123)
                    # ID가 항상 'PREFIX_숫자' 형식임을 가정합니다.
                    numeric_part = extracted_id.split('_')[-1]
                    numeric_id = int(numeric_part)
                    
                    # 3. 데이터 리스트에 추가 (정렬을 위한 임시 컬럼 포함)
                    mapping_data.append({
                        'ID': extracted_id,
                        '파일명': filename,
                        'SortIndex': numeric_id
                    })
                    mapped_count += 1
                
            except Exception as e:
                print(f"  ❌ 파일 분석 실패 ({filename}): {e}")

    if not mapping_data:
        print(f"⚠️ {prefix} 패턴에 일치하는 파일을 찾지 못했습니다. CSV 파일을 생성하지 않습니다.")
        return 0

    # 4. DataFrame 생성, 정렬 및 CSV 저장
    df = pd.DataFrame(mapping_data)
    
    # SortIndex(숫자)를 기준으로 오름차순 정렬
    df = df.sort_values(by='SortIndex', ascending=True)
    
    # 임시로 사용한 SortIndex 컬럼 제거
    df = df.drop(columns=['SortIndex'])
    
    try:
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"✅ ID 매핑 CSV 파일 생성 및 정렬 완료: {output_file}")
        print(f"   총 {mapped_count}개의 행이 기록되었습니다.")
        return mapped_count
    except Exception as e:
        print(f"🚨 CSV 파일 저장 중 오류 발생: {e}")
        return 0


# --- 메인 실행 ---
if __name__ == "__main__":
    start_time = time.time()
    
    # 2단계: 분양(sale) 파일 처리
    total_count = create_mapping_csv(
        DOWNLOAD_DIR, 
        PREFIX_TO_MAP, 
        OUTPUT_CSV_FILE
    )
    
    end_time = time.time()
    print("\n============================================")
    print("✨ ID 매핑 CSV 생성 작업 완료")
    print(f"처리된 파일 수: {total_count}개")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print("============================================")

--- 1. LH_sale_ 파일 ID 매핑 작업 시작 (정렬 포함) ---
✅ ID 매핑 CSV 파일 생성 및 정렬 완료: lh_sale_notices-download.csv
   총 412개의 행이 기록되었습니다.

✨ ID 매핑 CSV 생성 작업 완료
처리된 파일 수: 412개
총 소요 시간: 0.01초


---

### 고유값 몇 개인지 확인

In [63]:
import pandas as pd
import os
import time

# ----------------------------------------------------
# 🌟 설정 변수
# ----------------------------------------------------
LEASE_CSV_FILE = "lh_lease_notices_eng.csv"
SALE_CSV_FILE = "lh_sale_notices_eng.csv"

# ----------------------------------------------------

def analyze_regions(file_path, notice_type):
    """
    CSV 파일을 로드하여 '지역' 컬럼의 고유값과 개수를 분석하여 출력합니다.
    """
    print(f"\n--- {notice_type} 공고 ({file_path}) 지역 분석 시작 ---")
    
    if not os.path.exists(file_path):
        print(f"🚨 오류: 파일을 찾을 수 없습니다: {file_path}")
        return

    try:
        # 1. 파일 로드
        df = pd.read_csv(file_path)
        
        # 2. '지역' 컬럼 확인
        if '지역' not in df.columns:
            print(f"🚨 오류: '{file_path}' 파일에 '지역' 컬럼이 없습니다. 컬럼 목록: {list(df.columns)}")
            return
        
        # 3. '지역' 컬럼의 고유값 및 개수 계산 (value_counts 사용)
        region_counts = df['지역'].value_counts()
        
        # 4. 결과 출력
        print(f"✅ {notice_type} 공고 지역별 고유값 및 개수:")
        # to_string(header=True)를 사용하여 깔끔하게 출력
        print(region_counts.to_string(header=True, name=False))
        
        # 추가 정보: 고유 지역의 총 개수
        print(f"\n총 고유 지역 수: {len(region_counts)}개")

    except Exception as e:
        print(f"🚨 오류: {file_path} 파일 처리 중 예상치 못한 오류 발생: {e}")

# --- 메인 실행 ---
if __name__ == "__main__":
    start_time = time.time()
    
    # 1. 임대(Lease) 공고 분석
    analyze_regions(LEASE_CSV_FILE, "임대 (LEASE)")
    
    # 2. 분양(Sale) 공고 분석
    analyze_regions(SALE_CSV_FILE, "분양 (SALE)")
    
    end_time = time.time()
    print("\n============================================")
    print("✨ LH 공고 지역 분석 작업 완료")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print("============================================")


--- 임대 (LEASE) 공고 (lh_lease_notices_eng.csv) 지역 분석 시작 ---
✅ 임대 (LEASE) 공고 지역별 고유값 및 개수:
지역
경기도        340
경상남도       168
전북특별자치도    158
충청북도       110
인천광역시      105
강원특별자치도     99
충청남도        89
전라남도        77
경상북도        76
광주광역시       57
부산광역시       50
서울특별시       50
대전광역시       35
대구광역시       34
전국          32
울산광역시       31
대구광역시 외     30
제주특별자치도     24
인천광역시 외     22
세종특별자치시     14
부산광역시 외     12
광주광역시 외     11
대전광역시 외     10
서울특별시 외      9

총 고유 지역 수: 24개

--- 분양 (SALE) 공고 (lh_sale_notices_eng.csv) 지역 분석 시작 ---
✅ 분양 (SALE) 공고 지역별 고유값 및 개수:
지역
경기도        70
경상남도       14
제주특별자치도    12
경상북도        8
광주광역시       6
부산광역시       6
전라남도        6
전국          6
울산광역시       6
전북특별자치도     5
세종특별자치시     5
강원특별자치도     5
인천광역시       4
서울특별시       4
대구광역시       3
대전광역시       2
충청남도        1
충청북도        1

총 고유 지역 수: 18개

✨ LH 공고 지역 분석 작업 완료
총 소요 시간: 0.02초


- LH_임대에서 서울/경기 지역만 추출해서 core.csv 파일로 저장

In [89]:
import pandas as pd
import os
import shutil

# 1. 설정 변수
# --- 1단계 (ID 추출 소스) ---
MASTER_CSV_FILE = "lh_lease_notices_eng.csv"
MASTER_OUTPUT_FILE = "lh_lease_notices_eng_core.csv"
REGION_FILTER = ['서울', '경기'] # 필터링할 지역 목록
ID_COLUMN_NAME = 'ID'

# --- 2단계 (추출된 ID로 필터링할 파일) ---
DOWNLOAD_CSV_FILE = "lh_lease_notices-download.csv"
DOWNLOAD_OUTPUT_FILE = "lh_lease_notices-download_core.csv"

# --- 3단계 (파일 시스템 필터링 및 복사) ---
SOURCE_DIR = "lh_downloads"
TARGET_DIR = "lh_downloads_core"
# ★★★ 중요 ★★★
# 2단계 결과 CSV 파일에 실제 다운로드된 파일명이 저장된 컬럼 이름입니다.
# 만약 '파일명'이 아니면 해당 컬럼 이름으로 수정하세요.
FILE_NAME_COLUMN = '파일명' 


def check_and_load_csv(file_name, required_columns):
    """CSV 파일의 존재 여부와 필수 컬럼 포함 여부를 확인하고 DataFrame을 반환합니다."""
    if not os.path.exists(file_name):
        print(f"🚨 오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {file_name}")
        return None

    try:
        df = pd.read_csv(file_name)
    except Exception as e:
        print(f"🚨 오류: {file_name} 파일을 로드하는 중 문제가 발생했습니다: {e}")
        return None

    # 필수 컬럼 존재 여부 확인
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"🚨 오류: {file_name}에 필수 컬럼 {missing_columns}이(가) 존재하지 않습니다.")
        print(f"현재 컬럼 목록: {list(df.columns)}")
        return None

    return df

def copy_files_using_csv_filenames(core_csv_path):
    """
    주어진 CSV 파일의 '파일명' 컬럼에 있는 목록을 기준으로 
    SOURCE_DIR에서 파일을 찾아 TARGET_DIR로 복사합니다.
    """
    print(f"\n📌 단계 3 시작: {core_csv_path}의 '{FILE_NAME_COLUMN}' 컬럼 파일 목록을 기준으로 파일 복사")
    
    # 1. 대상 디렉토리 생성
    os.makedirs(TARGET_DIR, exist_ok=True)
    print(f"✅ 대상 폴더 준비 완료: {TARGET_DIR} (이미 존재하면 건너뜀)")

    # 2. 소스 CSV 파일 로드 및 파일명 리스트 추출
    df_core = check_and_load_csv(core_csv_path, [FILE_NAME_COLUMN])
    if df_core is None:
        print("🚨 오류: 2단계 결과 CSV 파일 로드 실패 또는 파일명 컬럼 없음. 3단계 파일 복사를 건너뜜.")
        return

    # CSV 파일에서 실제 파일명 리스트 추출 (문자열로 변환하고, 혹시 모를 중복 제거)
    target_filenames = df_core[FILE_NAME_COLUMN].astype(str).dropna().unique().tolist()
    
    total_target_count = len(target_filenames)
    print(f"🔍 2단계 CSV에서 추출된 대상 파일명 수: {total_target_count:,}개.")
    
    # 3. 소스 디렉토리 존재 여부 확인
    if not os.path.isdir(SOURCE_DIR):
        print(f"🚨 오류: 소스 폴더를 찾을 수 없습니다. 경로를 확인해주세요: {SOURCE_DIR}")
        return

    copied_count = 0
    missing_count = 0
    
    # 4. 파일명 리스트를 순회하며 복사
    print("\n🔎 파일 복사 진행 상황:")
    for i, filename in enumerate(target_filenames):
        source_path = os.path.join(SOURCE_DIR, filename)
        target_path = os.path.join(TARGET_DIR, filename)

        if i < 5: # 처음 5개 파일에 대한 진행 상황 출력 (디버깅)
            print(f"  > [{i+1}/{total_target_count}] 대상 파일: {filename}", end='')

        if os.path.exists(source_path):
            try:
                if not os.path.exists(target_path):
                    shutil.copy2(source_path, target_path) 
                    copied_count += 1
                    if i < 5:
                        print(" -> ✅ 복사 완료")
                else:
                    if i < 5:
                        print(" -> ⚠️ 이미 존재함 (건너뜀)")
            except Exception as e:
                print(f"🚨 오류: 파일 복사 중 문제 발생 ({filename}): {e}")
        else:
            missing_count += 1
            if i < 5:
                print(" -> ❌ 소스 폴더에 없음")

    print(f"\n💾 파일 복사 완료: 총 {copied_count:,}개의 파일이 {TARGET_DIR}에 새로 복사되었습니다.")
    
    # 최종 결과 요약 출력
    print("\n==========================================")
    print(f"✅ 전체 작업 성공적으로 완료")
    print(f"🚀 CSV에 명시된 대상 파일 수: {total_target_count}개")
    print(f"🚀 소스 폴더에서 발견 및 복사된 파일 수: {copied_count}개")
    print(f"⚠️ 소스 폴더에 없어 누락된 파일 수: {missing_count}개")
    print("==========================================")


def process_pipeline():
    """
    전체 데이터 파이프라인을 실행합니다.
    """
    # ----------------------------------------------------
    # 단계 1: ID 추출 및 1차 파일 저장 (서울/경기 지역 기준)
    # ----------------------------------------------------
    print(f"📌 단계 1 시작: {MASTER_CSV_FILE}에서 ID 추출")
    
    df_master = check_and_load_csv(MASTER_CSV_FILE, ['지역', ID_COLUMN_NAME])
    if df_master is None:
        return

    extracted_ids = []
    try:
        filter_condition = df_master['지역'].astype(str).str.contains(
            '|'.join(REGION_FILTER),
            na=False
        )
        
        filtered_master_df = df_master[filter_condition]
        extracted_ids = filtered_master_df[ID_COLUMN_NAME].astype(str).tolist() 
        
        original_count = len(df_master)
        filtered_count = len(filtered_master_df)

        print(f"🔍 1차 필터링 완료: 총 {original_count:,}개 중 {filtered_count:,}개 추출됨 (조건: {REGION_FILTER})")

        filtered_master_df.to_csv(MASTER_OUTPUT_FILE, index=False, encoding='utf-8-sig')
        print(f"💾 1차 데이터 저장 완료: {MASTER_OUTPUT_FILE}에 {filtered_count:,}개의 행이 저장되었습니다.")
        
        if not extracted_ids:
            print("\n⛔ 1차 필터링 조건에 맞는 ID가 발견되지 않아 2, 3단계를 진행할 수 없습니다.")
            return

    except Exception as e:
        print(f"🚨 1단계 데이터 처리 중 예상치 못한 오류 발생: {e}")
        return

    # ----------------------------------------------------
    # 단계 2: 추출된 ID를 사용하여 2차 파일 필터링 및 저장
    # ----------------------------------------------------
    print(f"\n📌 단계 2 시작: {DOWNLOAD_CSV_FILE}에서 {len(extracted_ids):,}개 ID에 해당하는 행 필터링")
    
    # 2단계에서는 ID와 파일명 컬럼 모두 필요하다고 가정하고 로드합니다.
    df_download = check_and_load_csv(DOWNLOAD_CSV_FILE, [ID_COLUMN_NAME, FILE_NAME_COLUMN])
    if df_download is None:
        return

    try:
        second_filter_condition = df_download[ID_COLUMN_NAME].astype(str).isin(extracted_ids)
        
        filtered_download_df = df_download[second_filter_condition]
        
        original_download_count = len(df_download)
        filtered_download_count = len(filtered_download_df)
        
        print(f"🔍 2차 필터링 완료: 총 {original_download_count:,}개 중 {filtered_download_count:,}개 추출됨 (조건: 1차 ID 리스트)")

        filtered_download_df.to_csv(DOWNLOAD_OUTPUT_FILE, index=False, encoding='utf-8-sig')
        print(f"💾 2차 데이터 저장 완료: {DOWNLOAD_OUTPUT_FILE}에 {filtered_download_count:,}개의 행이 저장되었습니다.")
        
        if filtered_download_count == 0:
            print("\n⛔ 2단계 필터링 결과가 0행이므로 3단계 파일 복사를 건너뜁니다.")
            return

    except Exception as e:
        print(f"🚨 2단계 데이터 처리 중 예상치 못한 오류 발생: {e}")
        return
        
    # ----------------------------------------------------
    # 단계 3: 2단계 결과 파일을 기준으로 다운로드 파일 복사
    # ----------------------------------------------------
    copy_files_using_csv_filenames(DOWNLOAD_OUTPUT_FILE)


if __name__ == "__main__":
    process_pipeline()

📌 단계 1 시작: lh_lease_notices_eng.csv에서 ID 추출
🔍 1차 필터링 완료: 총 1,643개 중 399개 추출됨 (조건: ['서울', '경기'])
💾 1차 데이터 저장 완료: lh_lease_notices_eng_core.csv에 399개의 행이 저장되었습니다.

📌 단계 2 시작: lh_lease_notices-download.csv에서 399개 ID에 해당하는 행 필터링
🔍 2차 필터링 완료: 총 2,800개 중 785개 추출됨 (조건: 1차 ID 리스트)
💾 2차 데이터 저장 완료: lh_lease_notices-download_core.csv에 785개의 행이 저장되었습니다.

📌 단계 3 시작: lh_lease_notices-download_core.csv의 '파일명' 컬럼 파일 목록을 기준으로 파일 복사
✅ 대상 폴더 준비 완료: lh_downloads_core (이미 존재하면 건너뜀)
🔍 2단계 CSV에서 추출된 대상 파일명 수: 785개.

🔎 파일 복사 진행 상황:
  > [1/785] 대상 파일: (LH_lease_8)_남양주시지역국민임대주택예비입주자모집(2024.11.05공고)(마이마이서비스적용).pdf -> ✅ 복사 완료
  > [2/785] 대상 파일: (LH_lease_10)_남양주시지역국민임대주택예비입주자모집(2024.11.05공고)(마이마이서비스적용)(수정).pdf -> ✅ 복사 완료
  > [3/785] 대상 파일: (LH_lease_11)_24년_2차_든든전세주택_공급주택_정정2.xlsx -> ✅ 복사 완료
  > [4/785] 대상 파일: (LH_lease_11)_24년2차_든든전세주택_표준입주자모집공고문(통합공고)_정정2.pdf -> ✅ 복사 완료
  > [5/785] 대상 파일: (LH_lease_12)_24년_분양전환형_신혼신생아_공급주택.xlsx -> ✅ 복사 완료

💾 파일 복사 완료: 총 785개의 파일이 lh_downloads_core에 새로 복사되었습니다.

✅ 전체 작업 성공적으로 완료

- LH_분양에서 서울/경기 지역만 추출해서 core.csv 파일로 저장

In [90]:
import pandas as pd
import os
import shutil

# ==============================================================================
# 1. LH_SALE 데이터 처리를 위한 설정 변수
# ==============================================================================
# --- 1단계 (ID 추출 소스) ---
MASTER_CSV_FILE = "lh_sale_notices_eng.csv"
MASTER_OUTPUT_FILE = "lh_sale_notices_eng_core.csv"
REGION_FILTER = ['서울', '경기'] # 필터링할 지역 목록
ID_COLUMN_NAME = 'ID'

# --- 2단계 (추출된 ID로 필터링할 파일) ---
DOWNLOAD_CSV_FILE = "lh_sale_notices-download.csv"
DOWNLOAD_OUTPUT_FILE = "lh_sale_notices-download_core.csv"

# --- 3단계 (파일 시스템 필터링 및 복사) ---
SOURCE_DIR = "lh_downloads" # 다운로드된 파일들이 모두 모여있는 폴더
# ★★★ 요청하신 대로 LH_lease와 동일한 통합 폴더를 사용합니다. ★★★
TARGET_DIR = "lh_downloads_core" 
# 2단계 결과 CSV 파일에 실제 다운로드된 파일명이 저장된 컬럼 이름
FILE_NAME_COLUMN = '파일명' 
# ==============================================================================


def check_and_load_csv(file_name, required_columns):
    """CSV 파일의 존재 여부와 필수 컬럼 포함 여부를 확인하고 DataFrame을 반환합니다."""
    if not os.path.exists(file_name):
        print(f"🚨 오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {file_name}")
        return None

    try:
        # 인코딩 오류 방지를 위해 'utf-8-sig'로 시도
        df = pd.read_csv(file_name, encoding='utf-8-sig')
    except Exception as e:
        print(f"🚨 오류: {file_name} 파일을 로드하는 중 문제가 발생했습니다: {e}")
        return None

    # 필수 컬럼 존재 여부 확인
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"🚨 오류: {file_name}에 필수 컬럼 {missing_columns}이(가) 존재하지 않습니다.")
        print(f"현재 컬럼 목록: {list(df.columns)}")
        return None

    return df

def copy_files_using_csv_filenames(core_csv_path):
    """
    주어진 CSV 파일의 '파일명' 컬럼에 있는 목록을 기준으로 
    SOURCE_DIR에서 파일을 찾아 TARGET_DIR로 복사합니다.
    """
    print(f"\n📌 단계 3 시작: {core_csv_path}의 '{FILE_NAME_COLUMN}' 컬럼 파일 목록을 기준으로 파일 복사")
    
    # 1. 대상 디렉토리 생성
    os.makedirs(TARGET_DIR, exist_ok=True)
    print(f"✅ 대상 폴더 준비 완료: {TARGET_DIR} (LH_lease와 통합됨)")

    # 2. 소스 CSV 파일 로드 및 파일명 리스트 추출
    df_core = check_and_load_csv(core_csv_path, [FILE_NAME_COLUMN])
    if df_core is None:
        print("🚨 오류: 2단계 결과 CSV 파일 로드 실패 또는 파일명 컬럼 없음. 3단계 파일 복사를 건너뜜.")
        return

    # CSV 파일에서 실제 파일명 리스트 추출 (문자열로 변환하고, 혹시 모를 중복 제거)
    target_filenames = df_core[FILE_NAME_COLUMN].astype(str).dropna().unique().tolist()
    
    total_target_count = len(target_filenames)
    print(f"🔍 2단계 CSV에서 추출된 대상 파일명 수: {total_target_count:,}개.")
    
    # 3. 소스 디렉토리 존재 여부 확인
    if not os.path.isdir(SOURCE_DIR):
        print(f"🚨 오류: 소스 폴더를 찾을 수 없습니다. 경로를 확인해주세요: {SOURCE_DIR}")
        return

    copied_count = 0
    missing_count = 0
    skipped_existing = 0
    
    # 4. 파일명 리스트를 순회하며 복사
    print("\n🔎 파일 복사 진행 상황 (처음 5개 파일 표시):")
    for i, filename in enumerate(target_filenames):
        source_path = os.path.join(SOURCE_DIR, filename)
        target_path = os.path.join(TARGET_DIR, filename)

        # 처음 5개 파일에 대한 진행 상황 출력 (디버깅)
        if i < 5: 
            print(f"  > [{i+1}/{total_target_count}] 대상 파일: {filename}", end='')

        if os.path.exists(source_path):
            try:
                if not os.path.exists(target_path):
                    shutil.copy2(source_path, target_path) 
                    copied_count += 1
                    if i < 5:
                        print(" -> ✅ 복사 완료")
                else:
                    skipped_existing += 1
                    if i < 5:
                        print(" -> ⚠️ 이미 존재함 (건너뜀)")
            except Exception as e:
                print(f"🚨 오류: 파일 복사 중 문제 발생 ({filename}): {e}")
        else:
            missing_count += 1
            if i < 5:
                print(" -> ❌ 소스 폴더에 없음")

    print(f"\n💾 파일 복사 완료: 총 {copied_count:,}개의 파일이 {TARGET_DIR}에 새로 복사되었습니다.")
    
    # 최종 결과 요약 출력
    print("\n==========================================")
    print(f"✅ 전체 LH_SALE 작업 성공적으로 완료 (파일은 {TARGET_DIR}에 통합 저장됨)")
    print(f"🚀 CSV에 명시된 대상 파일 수: {total_target_count}개")
    print(f"🚀 소스 폴더에서 발견 및 새로 복사된 파일 수: {copied_count}개")
    print(f"🚀 (LH_lease에서 복사되었을 수 있는) 이미 존재하여 건너뛴 파일 수: {skipped_existing}개")
    print(f"⚠️ 소스 폴더에 없어 누락된 파일 수: {missing_count}개")
    print("==========================================")


def process_pipeline():
    """
    전체 LH_SALE 데이터 파이프라인을 실행합니다.
    """
    # ----------------------------------------------------
    # 단계 1: ID 추출 및 1차 파일 저장 (서울/경기 지역 기준)
    # ----------------------------------------------------
    print(f"📌 단계 1 시작: {MASTER_CSV_FILE}에서 ID 추출")
    
    df_master = check_and_load_csv(MASTER_CSV_FILE, ['지역', ID_COLUMN_NAME])
    if df_master is None:
        return

    extracted_ids = []
    try:
        filter_condition = df_master['지역'].astype(str).str.contains(
            '|'.join(REGION_FILTER),
            na=False
        )
        
        filtered_master_df = df_master[filter_condition]
        extracted_ids = filtered_master_df[ID_COLUMN_NAME].astype(str).tolist() 
        
        original_count = len(df_master)
        filtered_count = len(filtered_master_df)

        print(f"🔍 1차 필터링 완료: 총 {original_count:,}개 중 {filtered_count:,}개 추출됨 (조건: {REGION_FILTER})")

        filtered_master_df.to_csv(MASTER_OUTPUT_FILE, index=False, encoding='utf-8-sig')
        print(f"💾 1차 데이터 저장 완료: {MASTER_OUTPUT_FILE}에 {filtered_count:,}개의 행이 저장되었습니다.")
        
        if not extracted_ids:
            print("\n⛔ 1차 필터링 조건에 맞는 ID가 발견되지 않아 2, 3단계를 진행할 수 없습니다.")
            return

    except Exception as e:
        print(f"🚨 1단계 데이터 처리 중 예상치 못한 오류 발생: {e}")
        return

    # ----------------------------------------------------
    # 단계 2: 추출된 ID를 사용하여 2차 파일 필터링 및 저장
    # ----------------------------------------------------
    print(f"\n📌 단계 2 시작: {DOWNLOAD_CSV_FILE}에서 {len(extracted_ids):,}개 ID에 해당하는 행 필터링")
    
    df_download = check_and_load_csv(DOWNLOAD_CSV_FILE, [ID_COLUMN_NAME, FILE_NAME_COLUMN])
    if df_download is None:
        return

    try:
        second_filter_condition = df_download[ID_COLUMN_NAME].astype(str).isin(extracted_ids)
        
        filtered_download_df = df_download[second_filter_condition]
        
        original_download_count = len(df_download)
        filtered_download_count = len(filtered_download_df)
        
        print(f"🔍 2차 필터링 완료: 총 {original_download_count:,}개 중 {filtered_download_count:,}개 추출됨 (조건: 1차 ID 리스트)")

        filtered_download_df.to_csv(DOWNLOAD_OUTPUT_FILE, index=False, encoding='utf-8-sig')
        print(f"💾 2차 데이터 저장 완료: {DOWNLOAD_OUTPUT_FILE}에 {filtered_download_count:,}개의 행이 저장되었습니다.")
        
        if filtered_download_count == 0:
            print("\n⛔ 2단계 필터링 결과가 0행이므로 3단계 파일 복사를 건너뜁니다.")
            return

    except Exception as e:
        print(f"🚨 2단계 데이터 처리 중 예상치 못한 오류 발생: {e}")
        return
        
    # ----------------------------------------------------
    # 단계 3: 2단계 결과 파일을 기준으로 다운로드 파일 복사
    # ----------------------------------------------------
    copy_files_using_csv_filenames(DOWNLOAD_OUTPUT_FILE)


if __name__ == "__main__":
    process_pipeline()

📌 단계 1 시작: lh_sale_notices_eng.csv에서 ID 추출
🔍 1차 필터링 완료: 총 164개 중 74개 추출됨 (조건: ['서울', '경기'])
💾 1차 데이터 저장 완료: lh_sale_notices_eng_core.csv에 74개의 행이 저장되었습니다.

📌 단계 2 시작: lh_sale_notices-download.csv에서 74개 ID에 해당하는 행 필터링
🔍 2차 필터링 완료: 총 412개 중 186개 추출됨 (조건: 1차 ID 리스트)
💾 2차 데이터 저장 완료: lh_sale_notices-download_core.csv에 186개의 행이 저장되었습니다.

📌 단계 3 시작: lh_sale_notices-download_core.csv의 '파일명' 컬럼 파일 목록을 기준으로 파일 복사
✅ 대상 폴더 준비 완료: lh_downloads_core (LH_lease와 통합됨)
🔍 2단계 CSV에서 추출된 대상 파일명 수: 186개.

🔎 파일 복사 진행 상황 (처음 5개 파일 표시):
  > [1/186] 대상 파일: (LH_sale_2)_양주옥정A-4(1)BL공공분양주택추가입주자모집공고문(무이자할부)(241104)★.pdf -> ✅ 복사 완료
  > [2/186] 대상 파일: (LH_sale_2)_양주옥정A-4(1)BL동호배치표(241104).pdf -> ✅ 복사 완료
  > [3/186] 대상 파일: (LH_sale_2)_양주옥정A-4(1)BL팸플릿.pdf -> ✅ 복사 완료
  > [4/186] 대상 파일: (LH_sale_5)_팸플릿(의왕월암A-1및A-3블록공공분양)_최종.pdf -> ✅ 복사 완료
  > [5/186] 대상 파일: (LH_sale_5)_3-2.금융정보등(금융ㆍ신용ㆍ보험정보)제공동의서-작성방법.pdf -> ✅ 복사 완료

💾 파일 복사 완료: 총 186개의 파일이 lh_downloads_core에 새로 복사되었습니다.

✅ 전체 LH_SALE 작업 성공적으로 완료 (파일은 lh_downloads_core에 통